In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('E:\\data Science\\datasets\\Social_Network_Ads.csv')

In [3]:
df.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [4]:
df.shape

(400, 5)

In [5]:
df.isna().sum()

User ID            0
Gender             0
Age                0
EstimatedSalary    0
Purchased          0
dtype: int64

In [6]:
df.dtypes

User ID             int64
Gender             object
Age                 int64
EstimatedSalary     int64
Purchased           int64
dtype: object

In [7]:
df1 = df.drop(['User ID'],axis = 1)

In [8]:
df1.shape

(400, 4)

In [9]:
gender = pd.get_dummies(df1['Gender'])
gender.head()

,Female,Male
0,0,1
1,0,1
2,1,0
3,1,0
4,0,1


In [10]:
df1 = pd.concat([df1,gender],axis = 1)

In [11]:
df2 = df1.drop(['Gender'],axis = 1)

In [12]:
df2.head()

,Age,EstimatedSalary,Purchased,Female,Male
0,19,19000,0,0,1
1,35,20000,0,0,1
2,26,43000,0,1,0
3,27,57000,0,1,0
4,19,76000,0,0,1


In [13]:
x = df2.loc[:,['Age','EstimatedSalary']]

In [14]:
y = df1.loc[:,['Purchased']]

In [15]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state = 1234,test_size = 0.3)

In [16]:
from sklearn.tree import DecisionTreeClassifier

In [17]:
# build the model for decision tree
model = DecisionTreeClassifier(criterion = 'entropy',random_state = 1234).fit(x_train,y_train)

In [18]:
# predict y value for x test with the help of model
y_pred = model.predict(x_test)

In [19]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [20]:
accuracy_score(y_test,y_pred)*100

82.5

In [21]:
confusion_matrix(y_test,y_pred)

array([[68, 12],
       [ 9, 31]], dtype=int64)

In [22]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.88      0.85      0.87        80
           1       0.72      0.78      0.75        40

    accuracy                           0.82       120
   macro avg       0.80      0.81      0.81       120
weighted avg       0.83      0.82      0.83       120



In [86]:
########HYPERPARAMETER TUNING#################

In [23]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
para_ran = {'max_depth' : [4,None],
             'min_samples_leaf': randint(1,9),
             'criterion': ['gini', 'entropy']}

In [24]:
tree = DecisionTreeClassifier()

In [25]:
tree_cv = RandomizedSearchCV(tree,para_ran,cv = 5)

In [26]:
tree_cv.fit(x_train,y_train)

C:\Users\dpzsony\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=DecisionTreeClassifier(class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort=False,
                                                    random_state=None,
                                                    splitter='best')

In [27]:
print("tuned best parameter :{}".format(tree_cv.best_params_))

tuned best parameter :{'criterion': 'gini', 'max_depth': 4, 'min_samples_leaf': 8}


In [28]:
y1_pred = tree_cv.predict(x_test)

In [29]:
accuracy_score(y_test,y1_pred)

0.8666666666666667

In [30]:
confusion_matrix(y_test,y1_pred)

array([[70, 10],
       [ 6, 34]], dtype=int64)

In [31]:
print(classification_report(y_test,y1_pred))

              precision    recall  f1-score   support

           0       0.92      0.88      0.90        80
           1       0.77      0.85      0.81        40

    accuracy                           0.87       120
   macro avg       0.85      0.86      0.85       120
weighted avg       0.87      0.87      0.87       120



In [ ]:
################ TREE VISUALISATION ##################

In [32]:
from IPython.display import Image
from sklearn.externals.six import StringIO
from sklearn.tree import export_graphviz
import pydot

C:\Users\dpzsony\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [33]:
# to make the list of feature excluding the target feature
feature = list(df2.columns[0:2])
feature

['Age', 'EstimatedSalary']

In [35]:
dot_data = StringIO()
export_graphviz(tree_cv, out_file = dot_data, feature_names= feature, filled = True, rounded =True)
graph = pydot.graph_from_dot_data(dot_data.getvalue())
Image(graph[0].create_png())

NotFittedError: This RandomizedSearchCV instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.